### LangGraph Basics Overview

In [1]:
!pip install langgraph langchain anthropic langchain-anthropic

In [12]:
import sys , os
from typing import Annotated, List
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Get API Keys

In [3]:
sys.path.append("/content/drive/MyDrive/Colab Notebooks/")
from api_keys import _api_keys

#environment variables
os.environ["ANTHROPIC_API_KEY"] = _api_keys["ANTHROPIC_API_KEY"]
os.environ["LANGSMITH_API_KEY"] = _api_keys["LANGSMITH_API_KEY"]
!export LANGSMITH_TRACING=true

#model info
ANTHROPIC_MODEL='claude-3-haiku-20240307'

### START HERE
- https://langchain-ai.github.io/langgraph/#example

In [4]:
from typing import Literal
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

### V1: Basic Tool

In [16]:
# Define the tools for the agent to use (can also be defined and used with async)
@tool
def search(
    query: Annotated[str, "query you want to search the web for"]
    ) -> str:

    """Call to surf the web."""

    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


# Let's inspect some of the attributes associated with the tool.
print(search.name)
print(search.description)
print(search.args)

#tool supports parsing of annotations, nested schemas, etc...
print(search.args_schema.model_json_schema())

search
Call to surf the web.
{'query': {'description': 'query you want to search the web for', 'title': 'Query', 'type': 'string'}}
{'description': 'Call to surf the web.', 'properties': {'query': {'description': 'query you want to search the web for', 'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'search', 'type': 'object'}


### V2: Basic Tool with defined argument schema
##### Note: using pydantic model is explicit and simplifys notation required in function associated with schema

In [17]:
from pydantic import BaseModel, Field

#define pydantic argument schema to pass to tool for reference
class SearchInput(BaseModel):
    query: str = Field(description="query you want to search the web for")


# Define the tools for the agent to use (can also be defined and used with async)
@tool("web search tool", args_schema=SearchInput, return_direct=True)
def search(
    query: str
    ) -> str:

    """Call to surf the web."""

    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


# Let's inspect some of the attributes associated with the tool.
print(search.name)
print(search.description)
print(search.args)

#tool supports parsing of annotations, nested schemas, etc...
print(search.args_schema.model_json_schema())
print(search.return_direct)

web search tool
Call to surf the web.
{'query': {'description': 'query you want to search the web for', 'title': 'Query', 'type': 'string'}}
{'properties': {'query': {'description': 'query you want to search the web for', 'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'SearchInput', 'type': 'object'}
True


### V3: Tool defined using StructuredTool.from_function() instead of using @tool decorator

Note: StructuredTool class provides more configurability than @tool decorator

In [27]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

#define pydantic argument schema to pass to tool for reference
class SearchInput(BaseModel):
    query: str = Field(description="query you want to search the web for")


def search(
    query: str
    ) -> str:

    """Call to surf the web."""

    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

async def asearch(
    query: str
    ) -> str:

    """Call to surf the web."""

    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

#define the tool and its async implementation
search_tool = StructuredTool.from_function(
    func=search
    , name = 'WebSearch Tool'
    , description = 'Call to surf the web'
    , args_schema=SearchInput
    , return_direct=True
    , coroutine=asearch)

# Let's inspect some of the attributes associated with the tool.
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

#tool supports parsing of annotations, nested schemas, etc...
print(search_tool.args_schema.model_json_schema())
print(search_tool.return_direct)

#INVOKE IT (both sequential and async implementation used in unison)
print(search_tool.invoke({"query": "what is the weather in sf"}))
print(await search_tool.ainvoke({"query": "what is the weather in nyc"}))

WebSearch Tool
Call to surf the web
{'query': {'description': 'query you want to search the web for', 'title': 'Query', 'type': 'string'}}
{'properties': {'query': {'description': 'query you want to search the web for', 'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'SearchInput', 'type': 'object'}
True
It's 60 degrees and foggy.
It's 90 degrees and sunny.


In [6]:
#tools available for use
tools = [search]

#TOOL NODE
#This node acts as a bridge between the language model and the available tools.
#The state MUST contain a list of messages.
#The last message MUST be an `AIMessage`.
#The `AIMessage` MUST have `tool_calls` populated.
tool_node = ToolNode(tools)

#model to use in langraph calls
#bind_tools() ensures the model knows that is has tools available
model = ChatAnthropic(model=ANTHROPIC_MODEL
, temperature=0).bind_tools(tools)

In [7]:
# Define the function that determines whether to call a tool or return a message to the user
# aka determines weather to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:

    #get last message
    messages = state['messages']
    last_message = messages[-1]

    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"

    # Otherwise, we stop (reply to the user)
    return END

# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [8]:

# Define a new graph
# NOTE: MessagesState is a prebuilt state schema that has one attribute
# -- a list of LangChain Message objects, as well as logic for merging
# the updates from each node into the state.
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model) #agent, determines what (if any) actions to take
workflow.add_node("tools", tool_node) #tools, invokes tools if the agent decides to take an action

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge (make a decision)
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`. (dont make a decision, do what i say)
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the agent
# input_data: messages -> dictionary proviing the initital input to the workflow, (here we simulate a user asking a question that invokes a dummy tool)
# config: dictionary providing settings to run with workflow. here thread_id is useful for tracking or managing multiple conversations
final_state = app.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'The current weather in San Francisco is 60 degrees Fahrenheit with foggy conditions.'